In [1]:
import networkx as nx
from scipy import sparse
import numpy as np



def Connector(Q):
    D = nx.to_networkx_graph(Q,create_using=nx.DiGraph())
    Isolate_list=list(nx.isolates(D))
    if len(Isolate_list)>0:
        for i in Isolate_list:
            if i==0:
                Q[i+1,i]=0.0001
            else:
                Q[i-1,i]=0.0001
    del D
    return Q
#choose 

# index = 0
# Spike_train = neuron_spike[index]
# A = dy_list[index]
# mask_index = mask[:, :, int(mat_trigger[0 + index * 2, 0]):int(mat_trigger[1 + index * 2, 0])]
#A_shuffled = dy_list_shuffled[i]

Spike_train = np.load('spike train')
A = np.load('dynamic connection')
mask_index = np.load('mask_index')

#KNN for sparse
import networkx as nx
from scipy import sparse

N = A.shape[0]
np.fill_diagonal(A, 0)
# print(max(A[:,4]))
# A=np.where(A > 0.09, 1, 0)
k = 10

# W.sort(reverse=True)
B1 = np.zeros((N, N))
for i in range(N):
    W = sorted(A[i, :], reverse=True)
    #     print( W[k])
    B1[i, :] = np.where(A[i, :] > W[k], 1, 0)

# B=np.multiply(B1,A)
# print(W[k])
# print(A[20,1:20])
# print(B[20,1:20])


C1 = np.zeros((N, N))
for i in range(N):
    W = sorted(A[:, i], reverse=True)
    #     print( W[k])
    C1[:, i] = np.where(A[:, i] > W[k], 1, 0)
# C=np.multiply(C1,A)
Q1 = B1 + C1
Q2 = np.where(Q1 > .9, 1, 0)

Q = np.multiply(Q2, A)
# del A
del B1
del C1
del Q1
del Q2
Connector(Q)
D = nx.to_networkx_graph(Q, create_using=nx.DiGraph())
D.number_of_edges()

# adjacency = nx.to_scipy_sparse_array(A)
import networkx as nx
import scipy.sparse as sp
import pygenstability as pgs
import scipy.sparse as sp

adjacency = sp.csr_matrix(Q)
test = sp.csgraph.connected_components(adjacency)
all_results = pgs.run(adjacency,
                      min_scale=-1.5,
                      max_scale=1.5,
                      n_scale=200,
                      n_tries = 600,
                      constructor="directed")
all_results = pgs.identify_optimal_scales(all_results, kernel_size=14, window_size=14)
_ = pgs.plot_scan(all_results)
plt.show()
# with open('/Users/sonmjack/Downloads/simon_paper/markov/fam1_Markov_list_age10_'+ str(index) +'.pkl', 'wb') as f:
#     pickle.dump(all_results, f)
    
t_opt = 2
selected_partitions = all_results['selected_partitions']
Community = all_results['community_id'][selected_partitions[-t_opt]]
import numpy as np
import matplotlib.colors as mcolors
import random

Color_Code2 = ["red", "orange", "pink", "olive", "cyan", "black", "yellow", "green", "brown", "gray", "tomato",
               "violet", "yellowgreen", "y", "crimson", "darkgoldenrod", "darkmagenta", "indigo", "darkred",
               "darkkhaki", "orangered", 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanchedalmond',
               'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral',
               'cornflowerblue', 'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray',
               'darkkhaki', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darkseagreen', 'darkslateblue',
               'darkslategray', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue', 'dimgray', 'dodgerblue',
               'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite', 'goldenrod',
               'greenyellow', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory', 'khaki', 'lavender',
               'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan',
               'lightgoldenrodyellow', 'lightgreen', 'lightgray', 'lightpink', 'lightsalmon', 'lightseagreen',
               'lightskyblue', 'lightslategray', 'lightsteelblue', 'lightyellow']

list_color = []
for color in mcolors.CSS4_COLORS:
    list_color.append(color)
random.shuffle(list_color)
Color_Code1 = [*Color_Code2, *list_color]
Color_Code = [*Color_Code1, *list_color]
non_zero_indices_per_row = []
for row in Spike_train:
    non_zero_indices = np.where(row != 0)[0]
    non_zero_indices_per_row.append(list(non_zero_indices))
neuralData = []
ColorCode = []

it = 0
Nodes = range(len(non_zero_indices_per_row))
for i in range(max(Community) + 1):
    for m, j in zip(Nodes, range(N)):
        if Community[j] == i:
            non_zero_indices_per_row[int(m)].sort()
            neuralData.append(non_zero_indices_per_row[int(m)])
            ColorCode.append(Color_Code[i])
            # A_ordered_row[it,:]=Q[j,:]
            it += 1
# it=0
# for i in range(max(Community)+1):
#     for m,j in zip(Nodes,range(N)):
#         if Community[j]==i:
#             A_ordered[:,it]=A_ordered_row[:,j]
#             it+=1
plt.eventplot(neuralData, color=ColorCode)
# plot.xlim([8000,9000])
# plot.title('Spike raster plot')
# plt.title(r"%s,  $N_g=%d$" % (data[l],max(Community)+1), fontweight ='bold',)
# plt.title(r"%s,  $S_%d$,  $N_c=%d$" % (data[l],t_opt,max(Community)+1),fontsize = 22)
plt.title("Reordered neural spike trains " + f'N-{max(Community) + 1}')
plt.xlabel('Time', fontsize=13)
plt.ylabel('Rearranged Neuron ID', fontsize=13)
plt.yticks(fontsize=13)
plt.xticks(np.arange(0,20000, 5000), [str(int(i*5000/30)) for i in range(0, 4)],fontsize=13)
plt.show()
from matplotlib.colors import BoundaryNorm

pic = np.zeros((512, 512))
for i in range(mask_index.shape[2]):
    index_row = np.where(mask_index[:, :, i] == True)[0]
    index_con = np.where(mask_index[:, :, i] == True)[1]
    for j in range(len(index_row)):
        if Community[i] == 0:
            pic[index_row[j], index_con[j]] = np.max(Community) + 1
        else:
            pic[index_row[j], index_con[j]] = Community[i]
#cmap = sns.diverging_palette(220, 20, as_cmap=True)
import seaborn as sns

cmap = plt.get_cmap('tab10')  
bounds = np.linspace(-0.5, 8.5, 10)  
norm = BoundaryNorm(bounds, cmap.N)

sns.heatmap(pic, cmap=cmap, norm=norm)
plt.title('Community beasd on real data')
plt.show()
path = '/Users/sonmjack/Downloads/age10 result_fam1/wild_type_community.npy'
np.save(path,Community)





# Another part of hierarchical structure

import matplotlib.colors as colors
selected_partitions = all_results['selected_partitions']
for k in range(len(selected_partitions)):
    plt.figure(figsize=(8, 6))
    Community = all_results['community_id'][selected_partitions[k]]
    from matplotlib.colors import BoundaryNorm
    
    pic = np.zeros((512, 512))
    for i in range(mask_index.shape[2]):
        index_row = np.where(mask_index[:, :, i] == True)[0]
        index_con = np.where(mask_index[:, :, i] == True)[1]
        for j in range(len(index_row)):
            if Community[i] == 0:
                pic[index_row[j], index_con[j]] = np.max(Community) + 1
            else:
                pic[index_row[j], index_con[j]] = Community[i]
    #cmap = sns.diverging_palette(220, 20, as_cmap=True)
    import seaborn as sns
    
    tab10_colors = plt.cm.tab10(np.linspace(0, 1, 10))
    extra_colors = plt.cm.Set3(np.linspace(0, 1, 12))  # 使用不同的colormap生成额外颜色
    all_colors = np.vstack((tab10_colors, extra_colors[:12]))  # 只取额外颜色的前10种
    custom_cmap = colors.ListedColormap(all_colors)
    
    
    bounds = np.arange(0, 22) # 因为边界是从1开始，到12结束，所以需要加上13作为最后一个边界
    #cmap = plt.cm.get_cmap('Spectral', len(bounds)-1) # 获取颜色映射，'viridis'可以替换成你喜欢的colormap
    norm = BoundaryNorm(bounds, custom_cmap.N)
    
    
    sns.heatmap(pic,  cmap=custom_cmap, norm=norm)
    plt.title('Community beasd on real data')
    plt.savefig('/Users/sonmjack/Downloads/age10 result_fam1/mask/' + 'dynamic distence_' +f'{index}_' + f'{k}'+'.jpg')
    plt.close()


import plotly.graph_objects as go  # pragma: no cover
from plotly.offline import plot  # pragma: no cover
import sys

def plot_sankey(
    all_results,
    optimal_scales=True,
    live=False,
    filename="fig1.png",
    scale_index=None,
):  # pragma: no cover
    """Plot Sankey diagram of communities accros scale (plotly only).
    Args:
        all_results (dict): results from run function
        optimal_scales (bool): use optimal scales or not
        live (bool): if True, interactive figure will appear in browser
        filename (str): filename to save the plot
        scale_index (bool): plot scale of indices
    """
    sources = []
    targets = []
    values = []
    shift = 0

    if not scale_index:
        all_results["community_id_reduced"] = all_results["community_id"]
    else:
        all_results["community_id_reduced"] = [all_results["community_id"][i] for i in scale_index]

    community_ids = all_results["community_id_reduced"]
    if optimal_scales and ("selected_partitions" in all_results.keys()):
        community_ids = [community_ids[u] for u in all_results["selected_partitions"]]

 
# Community=all_results['community_id'][selected_partitions[-t_opt]]

    for i in range(len(community_ids) - 1):
        community_source = np.array(community_ids[i])
        community_target = np.array(community_ids[i + 1])
        source_ids = set(community_source)
        target_ids = set(community_target)
#         print(target_ids)
        for source in source_ids:
            for target in target_ids:
                value = sum(community_target[community_source == source] == target)
#                 print(community_target[community_source == source] == target)
                if value > 0:
                    values.append(value)
                    sources.append(source + shift)
                    targets.append(target + len(source_ids) + shift)
        shift += len(source_ids)

    layout = go.Layout(autosize=True)
    fig = go.Figure(
        data=[
            go.Sankey(
                node={
                    "pad": 1,
                    "thickness": 1,
                    "line": {"color": "black", "width": 0.0},
                },
                link={"source": sources, "target": targets, "value": values},
            )
        ],
        layout=layout,
    )

    plot(fig, filename=filename)
#     fig.update_layout(
#         title=data,
# #         xaxis_title="X Axis Title",
# #         yaxis_title="Y Axis Title",
# #         legend_title="Legend Title",
# #         font=dict(
# #             family="Courier New, monospace",
# #             size=18,
# #             color="RebeccaPurple")
#     )
    fig.update_layout(showlegend=False)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    
    Scale_enumerate=[r'$S_{%s}$' % (len(community_ids)-i)  for i in range(len(community_ids))]
    for x_coordinate, column_name in enumerate(Scale_enumerate):
        fig.add_annotation(
              x=x_coordinate,#Plotly recognizes 0-5 to be the x range.

              y=1.075,#y value above 1 means above all nodes
              xref="x",
              yref="paper",
              text=column_name,#Text
              showarrow=False,
              font=dict(
                  family="Tahoma",
                  size=16,
                  color="black"
                  ),
              align="left",
              )
    fig.write_image(filename)
    
#     if live:
#         fig.show()

#     return fig

plot_sankey(
    all_results,
    optimal_scales=True,
    live=False,
    filename='Sankey.png' ,
    scale_index=None,
)

FileNotFoundError: [Errno 2] No such file or directory: 'spike train'